## **Script para tratar os arquivos originais da base**

Esse processo não analisará o conteúdo dos dados, apenas nas raras excecões.

A finalidade aqui é preparar as informações para serem carregadas no outro Jupyter onde serão feitas as análises

O arquivo 2000_orig.txt o valor da ordem da música está depois do nome da banda/música

Os arquivos de 2001 a 2005 o valor da ordem da música está antes do nome da banda/música, e tem um "." (ponto) depois do valor

O arquivo de 2006 o valor da ordem da música está antes do nome da banda/música, e não tem um "." (ponto) depois do valor

In [1]:
import json
import os
import re
from collections import Counter

In [2]:
folder_original = "resources/bases/original/"
folder_threated = "resources/bases/tratada/"

In [3]:
def int_try_parse(value):
    try:
        return int(value), True
    except ValueError:
        return value, False
    except Exception:
        return value, False

In [104]:
def read_file_on_line(path_to_file: str):
    """
    Read a file and return one single line
    """
    with open(path_to_file) as f:
        return f.read()

In [105]:
def read_file_lines(path_to_file: str):
    """
    Read a file and return one single line
    """
    with open(path_to_file) as f:
        return [stripped for line in f if (stripped := line.strip())]

In [98]:
def threat_artist_with_hyphen_in_name(text):
    text = text.upper()
    text = text.replace("A-HA", "A HA")
    text = text.replace("B-52", "B 52")
    return text
    

## Tratando base **2000**

In [9]:
file = "2000_orig.txt"

In [10]:
path_to_file = f"{folder_original}{file}"
contents = read_file_on_line(path_to_file)

In [ ]:
print(contents)

In [43]:
stop_words = [
    "Repetiu",
    "Repetiu 457 e 319",
    "Problemas",
    "2."
]

In [44]:
words_list = contents.split() 

In [45]:
words_list = [word for word in words_list if word not in stop_words]

### Checar se tem número de ordem repetido

se tiver valor repetido, tem que procurar qual é a repetição e tratar como remover

In [59]:
list_order = [word for word in words_list if word.isnumeric()]
group_by_order = Counter(list_order)

print(all(c > 1 for c in group_by_order.values()))

False


In [ ]:
index_previous = 0

song_list = []

for index, word in enumerate(words_list[:100]):
    if word.isnumeric():             
        song_list.append(
            {
                "position": word,
                "artist": " ".join(words_list[index_previous:index]),
                "music": ""
            }
        )
        index_previous = index + 1

print(json.dumps(song_list, indent=4, sort_keys=True))

In [37]:
def save_new_file(items: list, year: int):
    path = f"{folder_threated}{year}_ok.txt"
    
    if os.path.exists(path):
        os.remove(path)
   
    with open(path, 'w') as f:
        for item in items:
            f.write(f'{item["position"]}|{item["artist"]}|{item["music"]}\n')

In [31]:
def validate_ammount(list_songs):
    return len(list_songs) == 500

## Arquivos de **2001 a 2004**

In [49]:
files = [
    "2001.txt",
    "2002.txt",
    "2003.txt",
    "2004.txt",
]
 
for file in files:
    path_to_file = f"{folder_original}{file}"
    contents = read_file_on_line(path_to_file)
    
    words_list = contents.split() 
    words_list = [word for word in words_list]
    
    pattern = "[\d]{1,}[\.][\s][\D]{1,}[-][\D]{1,}"
    song_list = []

    itens_in_list = re.findall(pattern, contents)
    
    for item in itens_in_list:
        point_position = item.find(".")
        order, _ = int_try_parse(item[:4].replace(".", ""))
        
        song_list.append(
                {
                    "position": order,
                    "artist": item[point_position + 1:].split("-")[0].strip().upper(),
                    "music": item[point_position + 1 :].split("-")[1].strip().upper()
                }
            )
        
    save_new_file(song_list, file[:4])
    print(f"Lista de {file} está como {validate_ammount(song_list)} tem {len(song_list)} registros")

Lista de 2001_orig.txt está como False tem 492 registros
Lista de 2002_orig.txt está como False tem 493 registros
Lista de 2003_orig.txt está como False tem 487 registros
Lista de 2004_orig.txt está como False tem 484 registros


## Arquivo **2005** 

In [ ]:
year = "2005"
path_to_file = f"{folder_original}{year}.txt"
contents = read_file_on_line(path_to_file)
pattern = "[\d]{1,}[\.][\s][\D]{1,}"
song_list = []

itens_in_list = re.findall(pattern, contents)

for item in itens_in_list:
    point_position = item.find(".")
    order, _ = int_try_parse(item[:4].replace(".", ""))

    song_list.append(
            {
                "position": order,
                "artist": item[point_position + 1:].strip().upper(),
                "music": ""
            }
        )

save_new_file(song_list, year)
print(f"Lista de {file} está como {validate_ammount(song_list)} tem {len(song_list)} registros")

### Identificar o nome da banda na lista e separar a música

Arquivos 2005 e 2006 os nomes dos artistas e da música não tem separador

## Arquivos de **2007 a 2021**

In [95]:
for year in range(2007, 2022, 1):
    path_to_file = f"{folder_original}{year}.txt"
    lines = read_file_lines(path_to_file)
    
    print(path_to_file, len(lines))
    song_list = []
    
    for item in lines:
        pattern = "([\d]{1,3})(.*)"
        itens_in_list = re.findall(pattern, item)
        
        # print(itens_in_list)
                
        order = itens_in_list[0][0]
        artist_music = itens_in_list[0][1].strip()
        
        # print(artist_music)
        
        if artist_music.startswith("-") or artist_music.startswith("."):
            artist_music = artist_music[1:].strip()
        
        try:
            artist = artist_music.split("-")[0].upper().strip()
            music = artist_music.split("-")[1].upper().strip()
        
        except IndexError:
            artist = "??"
            music = "??"
        
        song_list.append(
                {
                    "position": order,
                    "artist": artist,
                    "music": music
                }
            )
        
    save_new_file(song_list, year)
    print(f"Lista de {year} está como {validate_ammount(song_list)} tem {len(song_list)} registros")


resources/bases/original/2007.txt 501
Lista de 2007 está como False tem 501 registros
resources/bases/original/2008.txt 498
Lista de 2008 está como False tem 498 registros
resources/bases/original/2009.txt 500
Lista de 2009 está como True tem 500 registros
resources/bases/original/2010.txt 500
Lista de 2010 está como True tem 500 registros
resources/bases/original/2011.txt 500
Lista de 2011 está como True tem 500 registros
resources/bases/original/2012.txt 500
Lista de 2012 está como True tem 500 registros
resources/bases/original/2013.txt 499
Lista de 2013 está como False tem 499 registros
resources/bases/original/2014.txt 500
Lista de 2014 está como True tem 500 registros
resources/bases/original/2015.txt 500
Lista de 2015 está como True tem 500 registros
resources/bases/original/2016.txt 500
Lista de 2016 está como True tem 500 registros
resources/bases/original/2017.txt 500
Lista de 2017 está como True tem 500 registros
resources/bases/original/2018.txt 500
Lista de 2018 está como 